In [0]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM , Dropout , Dense

In [0]:
df_female = pd.read_csv('Female-Names.csv')
df_male = pd.read_csv('Male-Names.csv')

In [0]:
df_male.head(-5)

name gender    race
0                   barjraj      m  indian
1              ramdin verma      m  indian
2           sharat chandran      m  indian
3           birender mandal      m  indian
4                      amit      m  indian
5                    kushal      m  indian
6                     kasid      m  indian
7              shiv prakash      m  indian
8              vikram singh      m  indian
9                    sanjay      m  indian
10                     abhi      m  indian
11           ram dutt gupta      m  indian
12             khadak singh      m  indian
13             gurmit singh      m  indian
14               chanderpal      m  indian
15                     aman      m  indian
16                  khursid      m  indian
17                   rajeev      m  indian
18                  durgesh      m  indian
19              nahar singh      m  indian
20                ram kumar      m  indian
21              sunder paal      m  indian
22          maansingh aswal      m  indian
23                    rohit      m  indian
24                    rohit      m  indian
25                   sparsh      m  indian
26                  santosh      m  indian
27                  santosh      m  indian
28         punit khandelwal      m  indian
29                   dinesh      m  indian
...                     ...    ...     ...
14810         manish bansal      m  indian
14811          chander bhan      m  indian
14812                   jai      m  indian
14813  gurmeet singh @ golu      m  indian
14814                 bablu      m  indian
14815                 pinki      m  indian
14816        surender singh      m  indian
14817        vikram bahadur      m  indian
14818            kapil dutt      m  indian
14819                mukesh      m  indian
14820              harender      m  indian
14821           vinay kumar      m  indian
14822        surendra singh      m  indian
14823          bharat singh      m  indian
14824         vishal ahemad      m  indian
14825             ghanshyam      m  indian
14826                rayish      m  indian
14827                 ayush      m  indian
14828            sunder jha      m  indian
14829       bhupender singh      m  indian
14830         pradeep kumar      m  indian
14831           ajeet singj      m  indian
14832          rajesh kumar      m  indian
14833      ashok shrivastav      m  indian
14834         jarnail singh      m  indian
14835                  avid      m  indian
14836           vinod kumar      m  indian
14837             raj kumar      m  indian
14838        sanjya chhetri      m  indian
14839            raj kishor      m  indian

[14840 rows x 3 columns]

In [0]:
df = pd.concat([df_female , df_male])

In [0]:
char2idx = {}
char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
index = 0
for ch in char_set:
    char2idx[ch] = index
    index += 1
   

In [0]:
def should_keep(word):
    if len(word) > 19:
        return False
    for ch in word:
        if ch not in set(char_set):
            return False
    return True

In [0]:
def clean(word):
    word = str(word)
    char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
    name = word.lower()
    if should_keep(name):
        return name
    else:
        return None

In [0]:
df.name = df.name.apply(lambda x : clean(x))

In [0]:
df = df.dropna()

In [0]:
max_word_len = 20
def convert_one_hot(word):
    word_chars = []
    
    
    for ch in word:
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx[ch]] = 1
        word_chars.append(one_hot_vec)
    
    
    for _ in range(max_word_len-len(word)):
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx['END']] = 1
        word_chars.append(one_hot_vec)
    
    
    word_chars = np.array(word_chars)
    return word_chars

In [0]:
words = []
labels = []
for n , g in df.drop('race' , axis = 1).itertuples(index=False):
    words.append(convert_one_hot(n))
    labels.append(g)

In [0]:
words = np.array(words)
labels = np.array(labels)

In [0]:
words.shape

(27980, 20, 29)

In [0]:
ohe = OneHotEncoder()
y = ohe.fit_transform(labels.reshape(-1 , 1)).todense()

In [0]:
y[:10]

matrix([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [0]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

W0722 15:51:40.410032 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 15:51:40.452130 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 15:51:40.463095 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 15:51:40.913147 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 15:51:40.923230 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

W0722 15:51:41.275603 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 15:51:41.302454 139833879697280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0722 15:51:41.308983 139833879697280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
train_test_split?

In [0]:
model = Sequential()
model.add(LSTM(128 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20, 128)           80896     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 130,434
Trainable params: 130,434
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [0]:
train_test_split?

In [0]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [0]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(words, y, test_size=0.33, random_state=42)

In [0]:
model.fit(X_train , y_train , epochs = 10)

Epoch 1/10
18746/18746 [==============================] - 42s 2ms/step - loss: 0.5366 - acc: 0.7292
Epoch 2/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.4033 - acc: 0.8240
Epoch 3/10
18746/18746 [==============================] - 38s 2ms/step - loss: 0.3464 - acc: 0.8595
Epoch 4/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.3129 - acc: 0.8779
Epoch 5/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.2835 - acc: 0.8938
Epoch 6/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.2636 - acc: 0.9017
Epoch 7/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.2444 - acc: 0.9115
Epoch 8/10
18746/18746 [==============================] - 39s 2ms/step - loss: 0.2315 - acc: 0.9176
Epoch 9/10
18746/18746 [==============================] - 37s 2ms/step - loss: 0.2207 - acc: 0.9222
Epoch 10/10
18746/18746 [==============================] - 38s 2ms/step - loss: 0.2081 - acc: 0.9306

In [0]:
model.evaluate(X_test , y_test)

9234/9234 [==============================] - 7s 766us/step


[0.2314458807935595, 0.921810699536838]

In [0]:
name = 'shreya'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.9733752 , 0.02558519]], dtype=float32)

In [0]:
name = 'anuj'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.0723962, 0.9246639]], dtype=float32)

In [0]:
name = 'lovely'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.10951865, 0.8826611 ]], dtype=float32)

In [0]:
name = 'sweety'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.675811  , 0.33107132]], dtype=float32)

In [1]:
# first one indicates male and second one indicates female

In [5]:
#! git commit-m"Specified which one is male and which one is female"

In [4]:
#! git push -u origin master


In [ ]:
#Readme GENDER RECOGNITION PYTHON CODE